# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [85]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [86]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier.index import IterDictIndexer
import pandas as pd #extra import
from sklearn.feature_extraction.text import CountVectorizer #extra import

In [87]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [88]:
# Datenset laden
# Lade den Datensatz in test_dataset
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.

test_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print(test_dataset)

IRDSDataset('ir-lab-sose-2024/ir-acl-anthology-20240504-training')


Create a custom tokenizer method.
This method takes a String as an argument and returns a List of all tokens, including ngrams between 1 and 3 words.

In [89]:
# Methode, die einen String einliest und eine Liste aller ngrams in der Range ausgibt. 1grams: alle Wörter einzeln. 2grams: alle paare aus benachbarten etc.
def ngram_tokenizer(text, ngram_range=(1, 3)):
    vectorizer = CountVectorizer(ngram_range=ngram_range, token_pattern=r'\b\w+\b', analyzer='word')
    analyze = vectorizer.build_analyzer()
    return analyze(text)

#Beispiel, wie ein text getokenized wird
print(ngram_tokenizer("Testing of Information Retrieval Systems"));

['testing', 'of', 'information', 'retrieval', 'systems', 'testing of', 'of information', 'information retrieval', 'retrieval systems', 'testing of information', 'of information retrieval', 'information retrieval systems']


Create the Iterator for our dataset. This is an Iterator where every element is a dict which contains a text and a docno.
See the example print below.

In [90]:

# Erstelle einen Iterator von Dicts von unserem Dataset
# Return type : Iterator[Dict[str, Any]]
corpus_iter = test_dataset.get_corpus_iter()

#Print example element of corpus_iter.
#Corpus_iter is an iterator where all elements are dicts that look like this
for doc in corpus_iter:
     if doc.get("docno") == "L02-1310":
          print(doc)
       


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   8%|▊         | 9737/126958 [00:00<00:02, 49278.23it/s]

{'text': 'Bootstrapping Large Sense Tagged Corpora', 'docno': 'L02-1310'}


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 54531.28it/s]


In [91]:
# erstelle custom Indexer, der IterDictIndexer extended
class CustomTokenizerIterDictIndexer(IterDictIndexer):
    # Constructor, der den Konstruktor der Superklasse aufruft und die Argumente weitergibt
    def __init__(self, index_path, tokenizer):
        super().__init__(index_path) # übergebe Parameter an Konstruktor der Superklasse
        self.tokenizer = tokenizer  # Da man den tokenizer nicht als Parameter übergeben kann, muss man ihn so von hand ändern

    def _document_to_dict(self, doc):
        """
        Convert a document into a dictionary representation.
        Override this method to include custom tokenization.
        """
        if isinstance(doc, dict):
            doc_dict = doc
        else:
            doc_dict = {
                "docno": str(doc[0]),
                "text": doc[1]
            }
        
        # Tokenize the text using the custom tokenizer
        tokens = self.tokenizer(doc_dict["text"])
        
        # Create a dictionary with tokenized terms
        doc_dict["text"] = " ".join(tokens)
        return doc_dict

    def index(self, iter, fields=('text',), **kwargs):
        """
        Override the index method if necessary.
        """
        return super().index(iter, fields, **kwargs)




In [92]:
# Create an instance of the custom indexer
index_path = "./ngram_test_index1"
ngram_indexer = CustomTokenizerIterDictIndexer(index_path, ngram_tokenizer) 


#indexer = pt.IterDictIndexer('./index_ngram3', meta=['docno', 'text'], meta_lengths=[20, 4096])
#index_ref = indexer.index(transformed_corpus_iter, fields=['text'])

docs = test_dataset.get_corpus_iter()

docs =  [
    {"docno": "1", "text": "Information Retrieval Systems"},
    {"docno": "2", "text": "Systems Retrieval Information"}
]
index_ref = ngram_indexer.index(docs)   #,  fields=['text'])


index = pt.IndexFactory.of(index_ref)



ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]


In [153]:
print(test_dataset.get_topics('text'))

   qid                                     query
0    1  retrieval system improving effectiveness
1    2  machine learning language identification
2    3             social media detect self harm
3    4             stemming for arabic languages
4    5            audio based animal recognition
..  ..                                       ...
63  65         information in different language
64  66                  abbreviations in queries
65  67                  lemmatization algorithms
66  68                  filter ad rich documents
67  18     advancements in information retrieval

[68 rows x 2 columns]
Number of documents: 5
Number of terms: 4
Number of postings: 10
Number of fields: 1
Number of tokens: 16
Field names: [text]
Positions:   false



In [183]:

#index = tira.pt.index('ir-lab-sose-2024-needthegrade/baseline-retrieval-system/index_ngram2', test_dataset)

import pyterrier as pt
from pyterrier.index import IterDictIndexer


def custom_tokenizer(self, text):
        """
        Custom tokenizer method that lowercases and splits text by spaces.
        """
        vectorizer = CountVectorizer(ngram_range=(2,2), token_pattern=r'\b\w+\b', analyzer='word')
        analyze = vectorizer.build_analyzer()
        return analyze(text)
def ngram_tokenizer(text, ngram_range=(2,2)):
    vectorizer = CountVectorizer(ngram_range=ngram_range, token_pattern=r'\b\w+\b', analyzer='word')
    analyze = vectorizer.build_analyzer()
    return analyze(text)

class CustomTokenizerIterDictIndexer(IterDictIndexer):
    def __init__(self, index_path, overwrite, tokenizer):
        super().__init__(index_path, overwrite)
        self.tokenizer = tokenizer



    def get_text_dict(self, doc):
        text = doc['text']
        tokens = custom_tokenizer(text)
        return {'text': ' '.join(tokens)}
    

    def _document_to_dict(self, doc):
        """
        Convert a document into a dictionary representation.
        Override this method to include custom tokenization.
        """
        if isinstance(doc, dict):
            doc_dict = doc
        else:
            doc_dict = {
                "docno": str(doc[0]),
                "text": doc[1]
            }
        
        # Tokenize the text using the custom tokenizer
        tokens = self.custom_tokenizer(doc_dict["text"])
        
        # Create a dictionary with tokenized terms
        doc_dict["text"] = " ".join(tokens)
        return doc_dict

    def index(self, iter, fields=('text',), **kwargs):
        """
        Override the index method if necessary.
        """
        return super().index(iter, fields, **kwargs)

# Initialize PyTerrier


# Create an instance of the custom indexer
index_path = "./my_custom_index"
custom_indexer = CustomTokenizerIterDictIndexer(index_path, overwrite = True, tokenizer=ngram_tokenizer)

# Example documents to index
docs = [
    {"docno": "0", "text": "information retrieval systems for about a million times i try to do the system for the retrieval pipeline, why doesnt it work"},
    {"docno": "1", "text": ""}
]

# Index the documents
index_ref = custom_indexer.index(docs)

# Now you can use the index_ref as usual
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics())
for term, le in index.getLexicon():
    print(term) 
    print(le.getFrequency())

# Access the MetaIndex and Lexicon
meta = index.getMetaIndex()
lexicon = index.getLexicon()
inverted = index.getInvertedIndex()
print(inverted)
print(inverted)


# Print indexed terms for each document
for doc in docs:
    doc_id = meta.getDocument("docno", doc["docno"])
    print(f"Document {doc['docno']} index entries:")
    for termid in index.getInvertedIndex().getLexicon().getLexiconEntry(termid):
        term = lexicon.getLexiconEntry(termid).getKey()
        freq = index.getDocumentIndex()
        

15:29:23.940 [ForkJoinPool-41-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1) - further warnings are suppressed
15:29:24.179 [ForkJoinPool-41-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
Number of documents: 2
Number of terms: 8
Number of postings: 8
Number of fields: 1
Number of tokens: 10
Field names: [text]
Positions:   false

doesnt
1
million
1
pipelin
1
retriev
2
system
2
time
1
try
1
work
1
<org.terrier.structures.PostingIndex at 0x7b6ea6033a10 jclass=org/terrier/structures/PostingIndex jself=<LocalRef obj=0x5e3bfdd9e1d8 at 0x7b6ea786ef90>>


TypeError: 'org.terrier.structures.PostingIndex' object is not iterable

### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [145]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

### Step 4: Create the Run


In [146]:
print('First, we have a short look at the first three topics:')

test_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [152]:
print('Now we do the retrieval...')
#run = bm25(test_dataset.get_topics('text'))
run = bm25([{"qid": "0", "query" : "retrieval"}])
print('Done. Here are the first 10 entries of the run')
run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,docid,docno,rank,score,query
0,0,0,0,0,-0.394619,retrieval
1,0,4,4,1,-0.394619,retrieval
2,0,1,1,2,-0.573390,retrieval


In [136]:
#index = pt.IndexFactory.of(index_ref)
meta = index.getMetaIndex()

# List of metadata keys
meta_keys = ['docno', 'text']  # Adjust this list based on your meta fields

# Function to print document attributes
def print_doc_attributes(docid):
    attributes = {key: meta.getItem(key, docid) for key in meta_keys}
    print(f"Attributes for docid {docid}: {attributes}")

# Example: Print attributes for the first 5 documents
for docid in range(4):
    print_doc_attributes(docid)

# Function to print document attributes by docno
def print_doc_attributes_by_docno(docno):
    try:
        docid = meta.getDocument("docno", docno)
        print_doc_attributes(docid)
    except KeyError:
        print(f"Document with docno {docno} not found.")

# List of specific docnos to retrieve
#docnos = ['W05-0704']

# Retrieve and print attributes for each specified docno
#for docno in docnos:
 #   print_doc_attributes_by_docno(docno)

Attributes for docid 0: {'docno': '0', 'text': ''}
Attributes for docid 1: {'docno': '1', 'text': ''}
Attributes for docid 2: {'docno': '2', 'text': ''}
Attributes for docid 3: {'docno': '3', 'text': ''}


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [ ]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
